In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()),
    options = options
)

### Funciones y para qué sirven:

- El DOM (Document Object Model o Modelo de Objetos del Documento) es una estructura en forma de árbol que representa todo el contenido HTML y XML de una página web. Básicamente, cuando cargas una página, el navegador convierte el HTML en una estructura organizada de nodos (como elementos, atributos y texto) que se puede manipular desde JavaScript o herramientas como Selenium. Gracias al DOM, puedes localizar, leer o modificar cualquier parte de la página, como hacer clic en botones, leer textos de una tabla o llenar formularios de forma automática.

- webdriver.Chrome(): Lanza una nueva ventana del navegador Chrome. Esto inicia el entorno de automatización para interactuar con páginas web.

- driver.get(URL): Carga la página web cuyo enlace se proporciona. Es como si escribieras esa URL en la barra del navegador y dieras Enter.

- WebDriverWait(driver, tiempo).until(condición): Espera a que se cumpla una condición antes de seguir. Esto es crucial en páginas dinámicas que tardan en cargar ciertos elementos. Ayuda a que el script no falle por intentar acceder a algo que aún no ha aparecido.

- EC.presence_of_element_located((By.XXX, valor)): Verifica que un elemento esté presente en el DOM, que es la estructura jerárquica del HTML de la página. Aunque el elemento no se vea en pantalla todavía, esta condición se cumple si ya está cargado internamente.

- EC.element_to_be_clickable((By.XXX, valor)): Este método espera a que un elemento esté visible y habilitado, es decir, que esté listo para recibir clics. No basta con que esté en el DOM: debe ser visible en pantalla y no estar deshabilitado, oculto o bloqueado por otro elemento.

- driver.execute_script("JavaScript"): Permite ejecutar comandos JavaScript directamente sobre la página. Es útil cuando Selenium no puede interactuar con elementos de forma estándar (por ejemplo, botones escondidos o animaciones que bloquean clics).

- ActionChains(driver): Crea una “cadena de acciones” del ratón o teclado, como mover el puntero, hacer clic o arrastrar elementos. Es útil cuando necesitas simular una interacción más realista.

- .move_to_element(element).click().perform(): Mueve el ratón hasta un elemento en la pantalla y hace clic sobre él. Se usa cuando hay menús desplegables o botones que aparecen solo al pasar el ratón encima.

- element.click(): Realiza un clic sobre el elemento. Es más directo, pero a veces falla si el elemento está cubierto o fuera de la vista.

- driver.find_element() y driver.find_elements(): Estos métodos buscan uno o varios elementos dentro del DOM. Son la base para localizar cosas como botones, campos de texto, filas de una tabla, etc.

- element.scrollIntoView(): Hace que el navegador haga scroll hasta que el elemento esté visible. Esto es necesario para interactuar con elementos que no están inicialmente en la pantalla.

- element.scrollTop = element.scrollHeight: Esta línea simula un scroll dentro de un contenedor (por ejemplo, un div con una tabla). Es útil para forzar que se carguen más datos cuando la página usa carga diferida.

- element.text: Devuelve el texto que se ve en pantalla dentro de ese elemento del DOM. Por ejemplo, puede ser el nombre de un país o un valor numérico de la tabla.





In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Se inicializa el navegador Chrome usando Selenium WebDriver. Esto permite automatizar la interacción con una página web.

driver = webdriver.Chrome()

# Se abre la página de los datos del World Bank en el navegador. El comando get() es utilizado para cargar una URL.

driver.get("https://databank.worldbank.org/reports.aspx?source=2&series=FP.CPI.TOTL&country=")

# Se verifica si el banner de la página está presente. Si es así, se hace clic en el botón de cierre usando JavaScript.
# La espera explícita (WebDriverWait) asegura que el código no intente interactuar con el banner hasta que este cargue completamente.
# Si no se encuentra el banner, se maneja la excepción.

try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//input[@value='Close']"))
    )
    driver.execute_script("document.querySelector('input[value=\"Close\"]').click();")
    print("Banner cerrado")
except Exception as e:
    print("Error al cerrar el banner:", e)

# Esta es una espera de 3 segundos para garantizar que la página cargue completamente.
# Aunque no es ideal, se usa en este caso por simplicidad, ya que WebDriverWait solo se usa para condiciones específicas.

time.sleep(3)

# Se busca el botón desplegable de "Time" y se hace clic en él para abrir el menú.
# Se utiliza WebDriverWait para esperar hasta que el elemento esté clickeable.
# También se utiliza ActionChains para mover el cursor sobre el botón antes de hacer clic.

try:
    caret_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-text='Time']//span[@class='caret']"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", caret_button)
    actions = ActionChains(driver)
    actions.move_to_element(caret_button).click().perform()
    print("Desplegable de 'Time' abierto")
except Exception as e:
    print("Error al hacer clic en el caret:", e)

# Primero, se desmarcan todas las opciones del desplegable usando el botón "Unselect All Time".
# Luego se selecciona el checkbox de 2018, y finalmente se hace clic en "Apply Changes" para aplicar los cambios.
# La espera explícita se usa aquí también para asegurarse de que los elementos sean clickeables antes de interactuar con ellos.

try:
    unselect_all_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-text='UnSelect All Time']"))
    )
    unselect_all_button.click()
    print("Botón 'Unselect all' clickeado")
    
    checkbox = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.ID, "chk[WDI_Time].[Year].&[YR2018]"))
    )
    checkbox.click()
    print("Checkbox clickeado")
    
    apply_changes_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@class='actionBtn pull-right applyChangesDim' and @data-trackaction='apply changes']"))
    )
    apply_changes_button.click()
    print("Botón 'Apply Changes' clickeado")
except Exception as e:
    print("Error al interactuar con los botones:", e)

# Se espera 5 segundos para asegurarse de que los cambios hayan tenido tiempo
# de aplicarse antes de proceder con el desplazamiento de la página.

time.sleep(5)

# Se localiza el contenedor que contiene los datos (div con la clase dxgvCSD)
# y se desplaza hacia él para asegurarse de que sea visible.

dxgvCSD_div = driver.find_element(By.CLASS_NAME, "dxgvCSD")
driver.execute_script("arguments[0].scrollIntoView(true);", dxgvCSD_div)

# Se realiza un desplazamiento dentro del contenedor para cargar más datos, si están disponibles.
# El bucle permite desplazar hacia abajo varias veces para cargar más filas de la tabla.
# Se espera 2 segundos entre cada desplazamiento para dar tiempo a que se carguen los nuevos datos.

for _ in range(5):  # Ajusta el número de desplazamientos según sea necesario
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", dxgvCSD_div)
    time.sleep(2)  # Espera a que los datos se carguen

# Aquí, primero se espera a que la tabla sea visible. Luego, se recorren las filas de la tabla
# para extraer el país y el índice de confianza del consumidor (2018) de cada fila. Los datos
# se filtran para eliminar valores vacíos o inválidos. Finalmente, los datos se muestran con los
# encabezados "País" y "Índice de confianza del consumidor (2018)".

try:
    # Esperar a que la tabla sea visible
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//table[@id='grdTableView_DXMainTable']"))
    )

    # Obtener todas las filas de la tabla
    filas = driver.find_elements(By.XPATH, "//table[@id='grdTableView_DXMainTable']//tr")

    # Crear lista de datos
    datos = []
    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        if len(celdas) > 1:
            pais = celdas[0].text
            indice = celdas[1].text
            datos.append((pais, indice))

    # Filtrar y modificar los datos para mostrar el país como título y el índice con el nombre correcto
    datos_modificados = []
    for pais, numero in datos:
        if pais != '' and numero != '..':  # Filtrar los países sin datos
            datos_modificados.append((pais, numero))

    # Mostrar los encabezados de la tabla
    print("País", "Índice de confianza del consumidor (2018)")

    # Mostrar los datos modificados
    for pais, indice in datos_modificados:
        print(f"País: {pais}, Índice de confianza del consumidor (2018): {indice}")

except Exception as e:
    print("Error al extraer los datos de la tabla:", e)

# Se cierra el navegador después de haber extraído los datos necesarios para
# liberar los recursos del sistema y finalizar la automatización.
driver.quit()



Banner cerrado
Desplegable de 'Time' abierto
Botón 'Unselect all' clickeado
Checkbox clickeado
Botón 'Apply Changes' clickeado
País Índice de confianza del consumidor (2018)
País: Afghanistan, Índice de confianza del consumidor (2018): 146.5
País: Albania, Índice de confianza del consumidor (2018): 117.4
País: Algeria, Índice de confianza del consumidor (2018): 148.5
País: Angola, Índice de confianza del consumidor (2018): 324.2
País: Antigua and Barbuda, Índice de confianza del consumidor (2018): 113.8
País: Armenia, Índice de confianza del consumidor (2018): 127.3
País: Aruba, Índice de confianza del consumidor (2018): 105.1
País: Australia, Índice de confianza del consumidor (2018): 117.9
País: Austria, Índice de confianza del consumidor (2018): 116.3
País: Azerbaijan, Índice de confianza del consumidor (2018): 152.9
País: Bahamas, The, Índice de confianza del consumidor (2018): 113.4
País: Bahrain, Índice de confianza del consumidor (2018): 117.6
País: Bangladesh, Índice de confian

In [51]:
ICC = pd.DataFrame(datos_modificados)
ICC.columns = ["País", "Índice de confianza del consumidor (2018)"]
ICC

,País,Índice de confianza del consumidor (2018)
0,Afghanistan,146.5
1,Albania,117.4
2,Algeria,148.5
3,Angola,324.2
4,Antigua and Barbuda,113.8
...,...,...
78,Israel,107.3
79,Italy,110.0
80,Jamaica,156.4
81,Japan,105.0
